In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Load Data
input_tensor_train = np.load('/kaggle/input/wi-store/WI_arrary.npy') 
input_tensor_train = np.nan_to_num(input_tensor_train, nan=0.0)
input_tensor_train = np.where(input_tensor_train == -10, 0.0, input_tensor_train)

final_input = np.load('/kaggle/input/wi-store/WI_final_arrary.npy')
final_input = np.nan_to_num(final_input, nan=0.0)
final_input = np.where(final_input == -10, 0.0, final_input)

y = pd.read_parquet("/kaggle/input/wi-store/wi_target.parquet")
y = y.sort_values(['store_id', 'item_id', 'date'], ascending = [True, True, True])
y["store_item"] = y['store_id'] + '_' + y['item_id']
y.drop(columns=["store_id","item_id"], inplace = True)
y = y[y["date"] != "2014-04-24"]

target_tensor_train = y["sell units"].values
target_tensor_train = target_tensor_train.reshape(-1,28)
target_tensor_train = np.expand_dims(target_tensor_train, -1)

# Show shape
print(len(input_tensor_train), len(target_tensor_train))

# Decoder input creation (shifted targets with start token)
def create_decoder_inputs(target_tensor, start_token=0):
    batch_size = target_tensor.shape[0]
    start_tokens = np.full((batch_size, 1, 1), start_token, dtype=target_tensor.dtype)
    shifted = target_tensor[:, :-1, :]
    decoder_inputs = np.concatenate([start_tokens, shifted], axis=1)
    return decoder_inputs

decoder_inputs = create_decoder_inputs(target_tensor_train, start_token=0)
print(decoder_inputs.shape)
print("First decoder input sequence:\n", decoder_inputs[0].squeeze())

# Build the encoder–decoder model
latent_dim = 64

# Encoder
encoder_inputs = Input(shape=(input_tensor_train.shape[1], input_tensor_train.shape[2]))
encoder_lstm = LSTM(latent_dim, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs_layer = Input(shape=(decoder_inputs.shape[1], 1))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs_layer, initial_state=encoder_states)
decoder_dense = Dense(1)
decoder_outputs = decoder_dense(decoder_outputs)

# Define full model
model = Model([encoder_inputs, decoder_inputs_layer], decoder_outputs)
model.compile(optimizer='adam', loss='mse')
model.summary()

# Train the model
decoder_inputs = decoder_inputs.astype(np.float32)
target_tensor_train = target_tensor_train.astype(np.float32)

# Right-padding → no masking needed with CuDNN-compatible LSTM
history = model.fit([input_tensor_train, decoder_inputs], target_tensor_train,
                    batch_size=64,
                    epochs=100,
                    verbose=2)

# Inference models
# Encoder inference
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder inference
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_inputs_inf = Input(shape=(1, 1))  # One time step
decoder_lstm_inf = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs_inf, state_h_inf, state_c_inf = decoder_lstm_inf(
    decoder_inputs_inf, initial_state=decoder_states_inputs)
decoder_outputs_inf = decoder_dense(decoder_outputs_inf)

decoder_model = Model([decoder_inputs_inf] + decoder_states_inputs,
                      [decoder_outputs_inf, state_h_inf, state_c_inf])

# Inference decoding function
def decode_sequence(input_seq):
    input_tensor = tf.convert_to_tensor(input_seq, dtype=tf.float32)
    states_value = encoder_model(input_tensor, training=False)

    target_seq = tf.zeros((1, 1, 1), dtype=tf.float32)
    decoded = []

    for _ in range(28):
        output_tokens, h, c = decoder_model([target_seq] + states_value, training=False)
        predicted = output_tokens[0, 0, 0].numpy()
        decoded.append(predicted)
        target_seq = tf.convert_to_tensor([[[predicted]]], dtype=tf.float32)
        states_value = [h, c]

    return decoded

# Predict using inference
from tqdm import tqdm
predictions = []
for i in tqdm(range(len(final_input)), desc="Generating forecasts"):
    input_seq = final_input[i:i+1]
    pred = decode_sequence(input_seq)
    predictions.append(pred)

2025-05-03 21:42:47.470037: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746308567.711142      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746308567.781503      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


9147 9147
(9147, 28, 1)
First decoder input sequence:
 [0 '1' '1' '0' '0' '0' '2' '0' '1' '0' '2' '0' '0' '1' '0' '0' '0' '1' '0'
 '2' '0' '0' '1' '3' '1' '2' '0' '0']


I0000 00:00:1746308602.699167      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1746308602.699843      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 731, 33)        │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 28, 1)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ [(None, 64), (None,    │         25,088 │ input_layer[0][0]      │
│                           │ 64), (None, 64)]       │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, 28, 64),       │         16,896 │ input_layer_1[0][0],   │
│                           │ (None, 64), (None,     │                │ lstm[0][1], lstm[0][2] │
│                           │ 64)]                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 28, 1)          │             65 │ lstm_1[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

...

 Non-trainable params: 0 (0.00 B)

Epoch 1/100


I0000 00:00:1746308609.928789      99 cuda_dnn.cc:529] Loaded cuDNN version 90300


143/143 - 7s - 50ms/step - loss: 9.6859
Epoch 2/100
143/143 - 3s - 23ms/step - loss: 7.3621
Epoch 3/100
143/143 - 3s - 23ms/step - loss: 6.7791
Epoch 4/100
143/143 - 3s - 23ms/step - loss: 6.4197
Epoch 5/100
143/143 - 3s - 24ms/step - loss: 6.1541
Epoch 6/100
143/143 - 3s - 23ms/step - loss: 5.9237
Epoch 7/100
143/143 - 3s - 23ms/step - loss: 5.7493
Epoch 8/100
143/143 - 3s - 23ms/step - loss: 5.5890
Epoch 9/100
143/143 - 3s - 23ms/step - loss: 5.4575
Epoch 10/100
143/143 - 3s - 24ms/step - loss: 5.3316
Epoch 11/100
143/143 - 3s - 23ms/step - loss: 5.1938
Epoch 12/100
143/143 - 3s - 23ms/step - loss: 5.1252
Epoch 13/100
143/143 - 3s - 23ms/step - loss: 5.0027
Epoch 14/100
143/143 - 3s - 23ms/step - loss: 4.9888
Epoch 15/100
143/143 - 3s - 24ms/step - loss: 4.8630
Epoch 16/100
143/143 - 3s - 23ms/step - loss: 4.8188
Epoch 17/100
143/143 - 3s - 23ms/step - loss: 4.7422
Epoch 18/100
143/143 - 3s - 23ms/step - loss: 4.6993
Epoch 19/100
143/143 - 3s - 23ms/step - loss: 4.6826
Epoch 20/100
1

Generating forecasts: 100%|██████████| 9147/9147 [34:36<00:00,  4.40it/s]


In [2]:
pred_array = np.array(predictions) 

In [4]:
import numpy as np

np.save('/kaggle/working/wi_predict.npy', pred_array)